# MSCA ML Final Project: Face-to-BMI
# Part 2: Image Embedding and Basic VGG

Deliverables:

1-You must create a simple web API to predict a user's BMI in real-time. You can also use webcam input. The goal is to use one of the pre-trained image models (e.g. VGG Face), fine-tune with the provided data, and deploy via jupyter notebook, streamlit, flask or any other simple restful api's.

2-10 pages of the write-up about your implementation.

3-10 mins presentation or live demo in the final lecture.

Our goal is to beat the performance metrics provided in the paper.

## Set-up

In [ ]:
''' # colab mount
import os
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive/')
path_gdrive = '/content/drive/MyDrive/Colab Datasets/ML/BMI'
os.chdir(path_gdrive)
print(os.getcwd())'''

In [1]:
import os

# Google Bucket
bucket_path = 'gs://msca-sp23-bucket/ml_data'
file = 'BMI-20230313T174553Z-001.zip'
runtime_path = '/home/jupyter/data/ml/BMI'

os.chdir(runtime_path)
print(os.getcwd())

/home/jupyter/data/ml/BMI


In [2]:
import sys
import os
import pandas as pd
import numpy as np

import re
from tqdm import tqdm
tqdm.pandas()

from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)

In [4]:
# read csv file
bmi = pd.read_csv(runtime_path + '/all_data.csv')
bmi.head()

,bmi,gender,is_training,name,imgae_type,id,path
0,34.207396,Male,1,img_0.bmp,bmp,0,/home/jupyter/data/ml/BMI/Images/img_0.bmp
1,26.453720,Male,1,img_1.bmp,bmp,1,/home/jupyter/data/ml/BMI/Images/img_1.bmp
2,34.967561,Female,1,img_2.bmp,bmp,2,/home/jupyter/data/ml/BMI/Images/img_2.bmp
3,22.044766,Female,1,img_3.bmp,bmp,3,/home/jupyter/data/ml/BMI/Images/img_3.bmp
4,25.845588,Female,1,img_6.bmp,bmp,6,/home/jupyter/data/ml/BMI/Images/img_6.bmp


In [5]:
import tensorflow as tf
from tensorflow import keras

2023-05-15 23:53:44.695732: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-15 23:53:45.693129: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.12.0
Keras version: 2.12.0


In [ ]:
#!pip install keras_vggface

### Prepare Keras

In [2]:
# !python --version

Python 3.10.11


In [ ]:
# # @https://stackoverflow.com/questions/68862735/keras-vggface-no-module-named-keras-engine-topology
# ! pip install git+https://github.com/rcmalli/keras-vggface.git
# ! pip install keras_applications --no-deps

# #filename = "/usr/local/lib/python3.10/dist-packages/keras_vggface/models.py"
#filename = "/opt/conda/envs/nlp/lib/python3.10/site-packages/keras_vggface/models.py"
#text = open(filename).read()
#open(filename, "w+").write(text.replace('keras.engine.topology', 'from keras.utils.layer_utils'))

## Extact Features

1. Read images with Kreas API: https://machinelearningmastery.com/how-to-load-convert-and-save-images-with-the-keras-api/

In [11]:
# fixing a bug here: https://github.com/rcmalli/keras-vggface/issues/73

In [10]:
import tensorflow as tf
from keras_vggface.vggface import VGGFace
from keras.models import Model
from keras.preprocessing.image import *

In [12]:
bmi.head()

,bmi,gender,is_training,name,imgae_type,id,path
0,34.207396,Male,1,img_0.bmp,bmp,0,/home/jupyter/data/ml/BMI/Images/img_0.bmp
1,26.453720,Male,1,img_1.bmp,bmp,1,/home/jupyter/data/ml/BMI/Images/img_1.bmp
2,34.967561,Female,1,img_2.bmp,bmp,2,/home/jupyter/data/ml/BMI/Images/img_2.bmp
3,22.044766,Female,1,img_3.bmp,bmp,3,/home/jupyter/data/ml/BMI/Images/img_3.bmp
4,25.845588,Female,1,img_6.bmp,bmp,6,/home/jupyter/data/ml/BMI/Images/img_6.bmp


In [13]:
train = bmi[bmi.is_training == 1]
val = bmi[bmi.is_training != 1]

In [14]:
# Initialize a model for feature extraction
vggface = VGGFace()

580070376/580070376 [==============================] - 4s 0us/step


In [15]:
# Match outputs to fc6 features
model = Model(inputs=vggface.input, outputs=vggface.get_layer('fc6').output)

# Define layer name for feature extraction
layer_name = 'fc6'

In [39]:
# example of loading an image with the Keras API
# since 2021 tensorflow updated the package and moved model directory
from tensorflow.keras.preprocessing.image import *
import keras_vggface.utils as utils

def extract_features(img_path):
    if os.path.exists(img_path):
        img = load_img(img_path, target_size=(224, 224))
        x = img_to_array(img)
        x = np.expand_dims(x, axis=0) # Add one dimension (channel)
        x = utils.preprocess_input(x, version=1)
        features = model.predict(x)
        return features.flatten()
    else:
        return None

In [40]:
# test = '/home/jupyter/data/ml/BMI/Images/img_0.bmp'
# extract_features(test)

1/1 [==============================] - 0s 310ms/step


array([ -1.4681633,  -0.5275203, -22.067028 , ..., -17.18492  ,
       -11.092118 ,  -0.3551265], dtype=float32)

Note:

1. Transfer Learning. https://medium.com/@leosimmons/estimating-body-mass-index-from-face-images-using-keras-and-transfer-learning-de25e1bc0212